## **Idea/Project recommendation system**

<div align="center">
<img src= "https://user-images.githubusercontent.com/78396437/178339280-5b31b149-dec1-40c2-aa26-9ccf555ef6d5.png" alt="Drone" align="center" width="390">
</div>

- **Name**: **Pratyush Singh**
- **Institute: Indian Institute of Technology (BHU) Varanasi**
- **Branch:Electrical Engineering**

#### **Loading Dependencies**

In [24]:
import pandas as pd
import numpy as np
import json
import re 
import sys
import itertools

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

In [25]:
%matplotlib inline

In [26]:
pd.set_option('display.max_columns', None)
pd.set_option("max_rows", None)

#### **1.Data Exploration**

In [27]:
idea_data = pd.read_csv('idea_data/idea_data.csv')

In [28]:
idea_data.head(10)

,Name,Project_Name,Topics,Year,Views,Likes,Popularity,Status,school/institute/job,Name_of_institute/school/job
0,Rohan Saha,Visualising and forecasting stocks using Dash,"['Dash python','Data visualisations','matplotl...",2016,107,37,71,1,2,NaN
1,Pradeep mishra,Multiplayer Game - Connect4,"['python','Pygame','Game Development','OOPs','...",2016,80,21,48,1,1,NaN
2,Preeti Sarwa,YouTube Transcript Summarizer,"['web development','HTML','CSS','javascript','...",2016,110,31,69,1,1,NaN
3,Abhinav Kumar,CLI Based Chat Tool,"['python','Socket Programming','Multithreading']",2016,156,12,35,4,1,NaN
4,Revanth,Instagram Automation Tool,"['web development','python','HTML','CSS','Sele...",2016,78,18,56,3,1,NaN
5,Ratan Dilkhush,Disease Prediction,"['machine learning','Django','CSS','HTML','web...",2016,93,37,72,2,1,NaN
6,Abhay Singh,Discord bot for stocks,"['Pandas','Plot.ly']",2016,32,15,38,2,0,NaN
7,Shanta Bedi,WhatsApp APK for stickers,"['Selenium','python','android']",2016,56,10,41,3,1,NaN
8,Abhay singh chauhan,Chatbot Song Recommender System,"['python','API','Chatbot','Machine Learning','...",2016,60,29,77,3,1,NaN
9,Raghav Krishna,Contact App Android,"['Java','android','SQLite']",2016,81,42,51,2,0,NaN


**First, lets checks whether or not `Topics` is actually in a list format:**

In [29]:
idea_data['Topics'][0] 

"['Dash python','Data visualisations','matplotlib','pandas','Machine Learning','web development','HTML','CSS','UI/UX']"

In [30]:
idea_data['Topics'][0][0] 

'['

**As we can see, it's actually a string format. So lets convert it to string format.**

In [31]:
idea_data['Topics'] = idea_data['Topics'].apply(lambda x: [re.sub(' ','_',i) for i in re.findall(r"'([^']*)'", x)])

In [32]:
idea_data['Topics'].values[0]

['Dash_python',
 'Data_visualisations',
 'matplotlib',
 'pandas',
 'Machine_Learning',
 'web_development',
 'HTML',
 'CSS',
 'UI/UX']

**Now we have the `Topics` column is in list format, which we will use for feature generation.**

In [33]:
idea_data.head(7)

,Name,Project_Name,Topics,Year,Views,Likes,Popularity,Status,school/institute/job,Name_of_institute/school/job
0,Rohan Saha,Visualising and forecasting stocks using Dash,"[Dash_python, Data_visualisations, matplotlib,...",2016,107,37,71,1,2,NaN
1,Pradeep mishra,Multiplayer Game - Connect4,"[python, Pygame, Game_Development, OOPs, DSA, ...",2016,80,21,48,1,1,NaN
2,Preeti Sarwa,YouTube Transcript Summarizer,"[web_development, HTML, CSS, javascript, pytho...",2016,110,31,69,1,1,NaN
3,Abhinav Kumar,CLI Based Chat Tool,"[python, Socket_Programming, Multithreading]",2016,156,12,35,4,1,NaN
4,Revanth,Instagram Automation Tool,"[web_development, python, HTML, CSS, Selenium,...",2016,78,18,56,3,1,NaN
5,Ratan Dilkhush,Disease Prediction,"[machine_learning, Django, CSS, HTML, web_deve...",2016,93,37,72,2,1,NaN
6,Abhay Singh,Discord bot for stocks,"[Pandas, Plot.ly]",2016,32,15,38,2,0,NaN


#### **2. Feature Engineering**

- **Normalize float and int variables**
- **Bucketing of Popularity variable to create 10 classes of values ranging from 0-10.**
- **One-hot-encoding of Year and Popularity Variables**
- **Create TF-IDF features of strings in list of column Topics**

In [34]:
idea_data['Popularity'].describe()

count    121.000000
mean      55.652893
std       26.068727
min        8.000000
25%       37.000000
50%       48.000000
75%       77.000000
max       99.000000
Name: Popularity, dtype: float64

In [35]:
idea_data['Year'].describe()

count     121.000000
mean     2019.107438
std         1.833947
min      2016.000000
25%      2018.000000
50%      2019.000000
75%      2020.000000
max      2022.000000
Name: Year, dtype: float64

In [36]:
# creating 10 point buckets for popularity
# Popularity is ranging from 0-100, so int division by 10 will create 10 different classes. 
idea_data['Popularity_bucket'] = idea_data['Popularity'].apply(lambda x: int(x/10))

In [37]:
#idea_data['Like ratio'] = idea_data['Likes']/idea_data['Views']

In [38]:
#feature_columns =['Year','Status','Popularity_bucket','school/institute/job','Like ratio']
feature_columns =['Year','Status','Popularity_bucket','school/institute/job']

In [39]:
def one_hot_encoding(df, column, new_name): 
    
        tf_df = pd.get_dummies(df[column])
        feature_names = tf_df.columns
        print(feature_names)
        tf_df.columns = [new_name + "|" + str(i) for i in feature_names]
        tf_df.reset_index(drop = True, inplace = True)    
        return tf_df

In [40]:
#function to build entire feature set
def create_feature_set(df, int_float_feature_cols):
    """ 
    Process idea data to create a final set of features that will be used to generate recommendations

    Parameters: 
        df (pandas dataframe): Idea data
        int_float_feature_cols (list(str)): List of float columns that will be scaled 
        
    Returns: 
        final: final set of features 
    """
    
    #tfidf Topics lists
    tfidf = TfidfVectorizer()
    tfidf_matrix =  tfidf.fit_transform(df['Topics'].apply(lambda x: " ".join(x)))
    topics_feature = pd.DataFrame(tfidf_matrix.toarray())
    topics_feature.columns = ['topic' + "|" + i for i in tfidf.get_feature_names()]
    topics_feature.reset_index(drop = True, inplace=True)

    #explicity_ohe = ohe_prep(df, 'explicit','exp')    
    year_ohe = one_hot_encoding(df, 'Year','year') * 0.15 #scalling down the parameter of year, because it is not as important as topic features
    popularity_ohe = one_hot_encoding(df, 'Popularity_bucket','pop') * 0.25 #scalling down the parameter of Popularity bucket, because it is not as important as topic features

    #scale feature columns
    features = df[int_float_feature_cols].reset_index(drop = True)
    scaler = MinMaxScaler()
    feature_scaled = pd.DataFrame(scaler.fit_transform(features), columns = features.columns) * 0.2 #scalling down these features, because it is not as important as topic features

    #concanenate all features
    final = pd.concat([topics_feature, feature_scaled, popularity_ohe, year_ohe], axis = 1)
     
    #adding project name and its topics to feature dataframe, so that human readable outputs could be accessed
    final['Project_Name']=df['Project_Name'].values
    final['Topics']=df['Topics'].values
    
    return final

In [41]:
complete_feature_set = create_feature_set(idea_data, feature_columns)

Int64Index([2016, 2017, 2018, 2019, 2020, 2021, 2022], dtype='int64')
Int64Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype='int64')


In [42]:
complete_feature_set.head(6)

,topic|3d_printing,topic|3d_scanning,topic|ai,topic|algorithm,topic|alogorithms,topic|analytics,topic|android,topic|angular,topic|anti,topic|api,topic|app,topic|app_development,topic|architectures,topic|arduino,topic|arm,topic|arrays,topic|asset,topic|astronomy,topic|astrophysics,topic|asynchronous_javascript,topic|authentication,topic|automation,topic|backtracking,topic|basic_mathematics,topic|basic_statistics,topic|beautiful_soup,topic|bfs,topic|bigintegers,topic|biology,topic|bitcoin,topic|blender,topic|blockchain,topic|bluetooth,topic|bootstrap,topic|bpe,topic|brand,topic|btc,topic|business,topic|cad,topic|camera,topic|chain,topic|chatbot,topic|chrome,topic|chrome_extension_api,topic|classification,topic|cloud,topic|cloud_firestore,topic|cnn,topic|coinbase,topic|colab,topic|company,topic|computer_vision,topic|context_api,topic|contracts,topic|counterfeit,topic|crypto,topic|css,topic|cuda,topic|dart,topic|das,topic|dash_python,topic|data_filtering,topic|data_handling,topic|data_preparation,topic|data_preprocessing,topic|data_processing,topic|data_structure_and_algorithm,topic|data_visualisation,topic|data_visualisations,topic|data_visualization,topic|dcnn,topic|deep_learning,topic|deserialization,topic|developer_tools,topic|digital_electronics,topic|display,topic|django,topic|django_rest_framework,topic|documentation,topic|dom,topic|dqn,topic|drone,topic|dsa,topic|dynamic_programming,topic|ecu,topic|education,topic|ejs,topic|electrocoagualiation,topic|electronics,topic|embedded_system,topic|ems,topic|encoder,topic|engineering,topic|esp32,topic|ethereum,topic|exception_handling,topic|express,topic|express_framework,topic|fabgl,topic|finance,topic|finderscope,topic|fine_tuning,topic|firebase,topic|firmware,topic|flask,topic|flutter,topic|framework,topic|gadjet,topic|game,topic|game_development,topic|game_engine,topic|gazebo,topic|gesture,topic|git,topic|gpu,topic|graph,topic|graphs,topic|gui,topic|hackathon,topic|hacking,topic|hand,topic|hardware,topic|health,topic|heroku,topic|html,topic|http,topic|hydraulics,topic|image_recognition,topic|imu,topic|infix_expression_evaluation,topic|infrared,topic|interface,topic|internet,topic|investment,topic|ios,topic|iot,topic|ir,topic|java,topic|java_awt,topic|java_swing,topic|javascript,topic|jdbc,topic|jetson,topic|jframe,topic|jpanel,topic|jquery,topic|jwt,topic|kali,topic|keras,topic|keyboard,topic|kotlin,topic|l298n,topic|learn,topic|linux,topic|lookup_tables,topic|lora,topic|lqr,topic|ly,topic|machine_learning,topic|markdown,topic|market,topic|marketing,topic|matlab,topic|matplotlib,topic|maven,topic|means_clustering,topic|mechanical,topic|mechanical_design,topic|meshlab,topic|meshmixer,topic|micro,topic|micro_controllers,topic|micro_processing,topic|middlewares,topic|mobile_app_development,topic|model,topic|model_implementation,topic|mongodb,topic|mqtt,topic|multithreading,topic|music,topic|mutual_funds,topic|mysql,topic|navigation_stack,topic|networking,topic|neural_network,topic|neural_networks,topic|nlp,topic|nodejs,topic|nsa,topic|obstacle_avoidance,topic|oops,topic|opencv,topic|orm_framework,topic|pandas,topic|path_finding,topic|path_planning,topic|phiro,topic|photogrammetry,topic|physics,topic|pid,topic|plot,topic|plotly,topic|plugin_development,topic|position_calculation,topic|predictions,topic|privacy,topic|problem_solving,topic|pybullet,topic|pygame,topic|pyowm,topic|python,topic|python_basics,topic|pytorch,topic|quadruped,topic|raspberry_pi,topic|react,topic|recognition,topic|recommendation,topic|recursion,topic|redux,topic|regression,topic|reinforcement_learning,topic|research,topic|rest_api,topic|restful_routing,topic|retro,topic|retropie,topic|rfe,topic|robot,topic|robotics,topic|ros,topic|router,topic|rviz,topic|satellites,topic|science,topic|seaborn,topic|security,topic|selenium,topic|senor,topic|sensor,topic|sensor_chains,topic|serialization,topic|server,topic|servo,topic|simulator,topic|sk,topic|slam,topic|smart,topic|smartpipe,topic|socket_communication,topic|soc

#### **3. User input featuring and printing results**

In [43]:
def user_input_processing(user_input):
    user_input_features = complete_feature_set[(complete_feature_set['Project_Name'] == user_input)]
    user_input_features=user_input_features.drop(['Project_Name','Topics'], axis = 1)
    return user_input_features


**Recommending function using cosine similarity**

<div align="center">
<img src= "https://user-images.githubusercontent.com/78396437/178341524-bdc3b55e-8a11-4f2a-9a84-6de8f46f7ad7.png" alt="Drone" align="center" width="390">
</div>

In [44]:
def recommend_songs(user_input,feature_idea_data,top_recommendations = 5):

    user_input_features=user_input_processing(user_input)

    #cosine similairity function:      
    feature_idea_data['similarity_ratio'] = cosine_similarity(feature_idea_data.drop(['Project_Name','Topics'], axis = 1).values, user_input_features.values[0].reshape(1, -1))[:,0]

    best_recommendations = feature_idea_data.sort_values('similarity_ratio',ascending = False)[['Project_Name','Topics']].iloc[1:top_recommendations+1]

    return best_recommendations

In [45]:
user_input='Time lock Wallet'
output_recommendations = recommend_songs(user_input,complete_feature_set,20)
output_recommendations.style.hide_index()

Project_Name,Topics
Savings and Lending Application,"['nodejs', 'blockchain', 'solidity', 'Ethereum']"
Multi-Send,"['solidity', 'Ethereum']"
Polling System,"['blockchain', 'web3', 'Ethereum']"
Digital Asset Marketplace (DAM),"['crypto', 'web3', 'react', 'Ethereum']"
Peer To Peer Carpooling,"['blockchain', 'python']"
Secure Voting System,"['blockchain', 'javascript']"
Decentralized Cryptocurrency Exchange,"['react', 'Angular', 'Django', 'blockchain', 'bitcoin', 'crypto', 'coinbase', 'btc']"
Blockchain Wallet,"['HTML', 'javascript', 'web3', 'blockchain', 'CSS']"
Fake Product Identification System,"['nodejs', 'mySQL', 'android', 'bootstrap', 'express', 'html', 'hackathon', 'CSS', 'Ethereum', 'supply-chain', 'solidity', 'anti-counterfeit']"
IoT Project-Learn to design an IoT Ready Infrastructure,"['IoT', 'MQTT', 'architectures', 'SmartPipe', 'Sensor_chains']"
